<a href="https://colab.research.google.com/github/girilang/COMP9414-Tutorials/blob/main/Tutorial_Week_9_Belief_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Artificial Intelligence - COMP9414 23T2
###Tutorial week 9 - Belief Networks

@Author: __Francisco Cruz__



##Belief Networks

A Bayesian or belief network is a probabilistic graphical model used to represent and analyze the dependencies between variables. In this network, nodes represent random variables, and directed edges between nodes indicate conditional dependencies. The strength of these dependencies is captured by conditional probability tables associated with each node.

Bayes Nets are widely used in various fields, including artificial intelligence, machine learning, statistics, and expert systems. They enable reasoning and inference by calculating probabilities based on evidence and updating beliefs in a systematic way using Bayesian probability principles.


##Belief Networks in Python

Consider the problem domain reviewed during the lecture when I go home [1], and I want to know if someone from my family is home before I go in. Let's say I know the following information:

1. When my wife leaves the house, she often (but not always) turns on the outside light.
2. When nobody is home, the dog is often left outside.
3. If the dog has bowel troubles, it is also often left outside.
4. If the dog is outside, I will probably hear it barking (though it might not bark, or I might hear a different dog barking and think it's my dog).

Given the previous information, we can consider the following five Boolean random variables:

1. family-out (fo): everyone is out of the house.
2. light-on (lo): the light is on.
3. dog-out (do): the dog is outside.
4. bowel-problem (bp): the dog has bowel troubles.
5. hear-bark (hb): I can hear the dog barking.

From this information, the following direct causal influences are appropriate:

1. hb is only directly influenced by do. Hence hb is conditionally independent of lo, fo and bp given do.
2. do is only directly influenced by fo and bp. Hence do is conditionally independent of lo given fo and bp.
3. lo is only directly influenced by fo. Hence lo is conditionally independent of do, hb and bp given fo.
4. fo and bp are independent.

Therefore, a belief network representing these direct causal relationships (though these causal connections are not absolute, i.e., they are not implications) can be built.
The network includes the prior probability of the random variable for root nodes fo and bp as well as the conditional probabilities of the node's variable given all possible combinations of its immediate parent nodes need to be determined for non-root nodes.

[1] Charniak, E. (1991). Bayesian networks without tears. AI magazine, 12(4), 50-50.

__Exercise__
First, we need to install pomegranate library.

In [1]:
#!pip install pomegranate==0.13.0
import pomegranate as pg

__Section 3.1.1.__ Using *pomegranate* create the root nodes fo and bp and assign them the respective prior probability.

In [ ]:
#section 3.1.1
fo = pg.DiscreteDistribution({'family out': 0.15, 'family home': 0.85})
bp = pg.DiscreteDistribution({'bowel problem': 0.01, 'no bowel problem': 0.99})

__Section 3.1.2.__ Create the non-root nodes lo, do, and hb. For lo and hb consider four combinations taking into account one immediate parent. For do consider eight combinations as this node has two immediate parents.

In [ ]:
#section 3.1.2
lo = pg.ConditionalProbabilityTable(
        [[ 'family home', 'light off', 0.95 ],
         [ 'family home', 'light on', 0.05 ],
         [ 'family out', 'light off', 0.4 ],
         [ 'family out', 'light on', 0.6 ]],[fo])

do = pg.ConditionalProbabilityTable(
        [[ 'family home', 'no bowel problem', 'dog out', 0.3 ],
         [ 'family home', 'bowel problem', 'dog out', 0.97 ],
         [ 'family out', 'no bowel problem', 'dog out', 0.9 ],
         [ 'family out', 'bowel problem', 'dog out', 0.99 ],
         [ 'family home', 'no bowel problem', 'dog in', 0.7 ],
         [ 'family home', 'bowel problem', 'dog in', 0.03 ],
         [ 'family out', 'no bowel problem', 'dog in', 0.1 ],
         [ 'family out', 'bowel problem', 'dog in', 0.01 ]],[fo,bp])

hb = pg.ConditionalProbabilityTable(
        [[ 'dog in', 'no hear bark', 0.99 ],
         [ 'dog in', 'hear bark', 0.01 ],
         [ 'dog out', 'no hear bark', 0.3 ],
         [ 'dog out', 'hear bark', 0.7 ]],[do])

__Section 3.1.3.__ Create the Bayesian network model.

In [ ]:
#section 3.1.3
model = pg.BayesianNetwork("MyBN")

__Section 3.1.4.__ Using the previous nodes, create five states and add them to the Bayesian network model.

In [ ]:
#section 3.1.4
FO = pg.State(fo, name="FO")
BP = pg.State(bp, name="BP")
LO = pg.State(lo, name="LO")
DO = pg.State(do, name="DO")
HB = pg.State(hb, name="HB")

model.add_states(FO, BP, LO, DO, HB)

__Section 3.1.5.__ Add to the model the four edges for the belief network.

In [ ]:
#section 3.1.5
model.add_edge(FO, LO)
model.add_edge(FO, DO)
model.add_edge(BP, DO)
model.add_edge(DO, HB)

__Section 3.1.6.__ Finalise the model to use it for joint and conditional probabilities.

In [ ]:
#section 3.1.6
model.bake()

__Section 3.1.7.__ Test the Bayesian network model for different situations.

* Test 1: Compute the joint probability P(\~FO,BP,~LO,DO,HB).
* Test 2: Compute a causal inference (top-down) using the conditional probability P(DO|BP).
* Test 3: Compute a diagnostic inference (bottom-up) using the conditional probability P(\~BP|\~DO).

In [ ]:
#section 3.1.7
#P(~FO,BP,~LO,DO,HB)
print(model.probability([["family home", "bowel problem", "light off", "dog out", "hear bark"]]))

#P(DO|BP)
print(model.predict_proba([{'BP': 'bowel problem'}])[0][3].parameters[0]['dog out'])

#P(~BP|~DO)
print(model.predict_proba([{'DO': 'dog in'}])[0][1].parameters[0]['no bowel problem'])

0.0054829250000000005
0.9729999999999995
0.9995531059138985
